# Decorators

## 3.7.5 Writing Your Own Decorator

In [12]:
from functools import wraps

def a_new_decorator(a_func):
    @wraps(a_func)
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_fun()
        print("I am doing some boring work after executing a_func()")
    return wrapTheFunction

@a_new_decorator
def a_function_requiring_decoration():
    """Hey yo! Decorate me!"""
    print("I am the function which needs some decoration to remove my foul smell.")
        
print(a_function_requiring_decoration.__name__)

a_function_requiring_decoration


### Blueprint:

In [13]:
from functools import wraps
def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args,**kwargs)
    return decorated

@decorator_name
def func():
    return("Function is running")

can_run = True
print(func())

Function is running


In [14]:
can_run = False
print(func())

Function will not run


Note: @wraps takes a function to be decorated and adds the functionality of copying over the function name, docstring, arguments list, etc. This allows access to the pre-decorated function's properties in the decorator.

Use-cases:

Now let's take a look at the areas where decorators really shine and their usage makes something really easy to manage.

1. Authorization - 
    Decorators can help to check whether someone is autorized to use an endpoint in a web application. They are extensively used in Flask and Django. here is an example to employ decorator based authentication:

In [15]:
from functools import wraps

def requires_auth(f):
    @wraps(f)
    def decorated(*args,**kwargs):
        auth = request.authorization
        if not auth or not check_auth(auth.username,auth.password):
            authenticate()
        return f(*args,**kwargs)
    return decorated

2. Logging - Logging is another area where decorators shine. Here is an example:

In [16]:
from functools import wraps

def logit(func):
    @wraps(func)
    def with_logging(*args,**kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return with_logging

@logit
def addition_func(x):
    """Do some math."""
    return x+x

result = addition_func(4)

addition_func was called


## 3.7.6 Decorators with Arguments

Nesting a Decorator Within a Function

Go back to the logging example, and create a wrapper which lets us specify a logfile to output to.

In [17]:
from functools import wraps

def logit(logfile='out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args,**kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # Open the logfile and append
            with open(logfile,'a') as opened_file:
                #Now we log to the specified logfile
                opened_file.write(log_string + '\n')
        return wrapped_function
    return logging_decorator
    
@logit()
def myfunc1():
    pass

myfunc1()

myfunc1 was called


In [18]:
@logit(logfile='func2.log')
def myfunc2():
    pass

myfunc2()

myfunc2 was called


Now we have our logit decorator in production, but when some parts of our application are considered critical, failure might be something that needs more immediate attention. Let's say sometimes you just want to log to a file. Other times you want an email sent, but still keep a log for your own records. This is a case for using inheritance, but so far we've only seen functions being used to build decorators.

Classes can also be used to build decorators. So, let's rebuild logit as a class instead of a function.

In [19]:
class logit(object):
    
    _logfile = 'out.log'
    
    def __init__(self,func):
        self.func = func
        
    def __call__(self,*args):
        log_string = self.func.__name__ + " was called"
        print(log_string)
        # Open the logfile and append
        with open(self._logfile, 'a') as opened_file:
            # Now we log to the specified logfile
            opened_file.write(log_string + '\n')
        # Now, send a notification
        self.notify()
        
        # return base func
        return self.func(*args)
    
    def notify(self):
        # logit only logs, no more
        pass

In [20]:
logit._logfile = 'out2.log'
@logit
def myfunc1():
    pass

myfunc1()

myfunc1 was called


Now, let's subclass logit to add email functionality

In [21]:
class email_logit(logit):
    '''
    A logit implementation for sending emails to admins
    when the function is called.
    '''
    def __init__(self,email='admin@myproject.com',*args,**kwargs):
        self.email = email
        super(email_logit,self).__init__(*args,**kwargs)
        
    def notify(self):
        # Send an email to self.email
        # Will not be implemented here
        pass

From here, @email_logit works just like @logit but sends an email to the admin in addition to logging.

## 3.8 Global & Return

Let's examine this little function:

In [23]:
def add(value1,value2):
    return value1 + value2

result = add(3,5)
print(result)

8


The function above takes two values as input and then outputs their sum. We could also have done:

In [24]:
def add(value1,value2):
    global result
    result = value1 + value2
    
add(3,5)
print(result)

8


The first function is assigning the value to the variable which calls the function (result = add(3,5)). In most cases you won't need to use the `global` keyword. 

The second function is making a global variable `result`. "Global" here means that we can access that variable outside the scope of the function as well.

In [25]:
# first without the global variable
def add(value1, value2):
    result = value1 + value2
    
add(2,4)
print(result)

8


In [26]:
def add(value1,value2):
    global result
    result = value1 + value2
    
add(2,4)
result

6

## 3.8.1 Multiple Return Values

In [27]:
def profile():
    name = "Danny"
    age = 30
    return name, age

profile_name, profile_age = profile()
print(profile_name)
print(profile_age)

Danny
30


Keep in mind that even in the above example we are returning a tuple (despite the lack of parenthesis) and not separate multiple values. To take it one step further, use `namedtuple`. Example:

In [28]:
from collections import namedtuple
def profile():
    Person = namedtuple('Person', 'name age')
    return Person(name="Danny", age =31)

p = profile()
print(p, type(p))

Person(name='Danny', age=31) <class '__main__.Person'>


In [29]:
print(p.name)
print(p.age)

Danny
31


In [30]:
p = profile()
print(p[0])
print(p[1])

Danny
31


In [31]:
name, age = profile()
print(name)
print(age)

Danny
31


## 3.9 Mutation

Mutable means 'able to be changed' and immutable means 'constant'. Consider this example:

In [32]:
foo = ['hi']
print(foo)

bar = foo
bar += ['bye']
print(foo)

['hi']
['hi', 'bye']


Whenever you assign a variable to another variable of mutable datatype, any changes to the data are reflected by both variables. The new variable is just an alias for the old variable. This is only true for mutable datatypes.

Another example involving functions and mutable data types:

In [35]:
def add_to(num, target=[]):
    target.append(num)
    return target

add_to(1)
add_to(2)
add_to(3)

[1, 2, 3]

In Python the default arguments are evaluated once the function is defined, not each time the function is called. Never define default arguments of mutable type unless you know what you are doing. Do something like this instead:

In [36]:
def add_to(element, target=None):
    if target is None:
        target = []
        target.append(element)
        return target

Now, whenever you call the function without the `target` argument, a new list is created. For instance:

In [37]:
add_to(42)

[42]

In [38]:
add_to(42)

[42]

In [39]:
add_to(42)

[42]

## 3.10 `__slots__` Magic

In Python, every class can have instance attributes. By default, Python uses a dict to store an object's instance attributes. This allows setting arbitrary new attributes at runtime.

However, for small classes with known attributes, it might be a bottleneck. The dict wastes a lot of memory. Python can't just allocate a static amount of memory at object creation to store all attributes. Therefore, it takes a lot memory if you create a lot of objects (in the >1,000 range). 

This issue can be circumvented by using `__slots__` to tell Python not to use a dict, and only allocate space for a fixed set of attributes. Here is an example with and without `__slots__`.

**Without `__slots__`:**

In [ ]:
class MyClass(object):
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
        self.set_up()
    # ...

**With `__slots__`:**

In [ ]:
class MyClass(object):
    __slots__ = ['name', 'identifier']
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
        set.set_up()
    # ...

The second piece of code will reduce the burden on your RAM.

On a sidenote, PyPy performs all of these optimizations by default.

Below you can see an example showing exact memory usage with and without `__slots__` done in IPython thanks to https://github.com/ianozsvald/ipython_memory_usage

In [41]:
import ipython_memory_usage.ipython_memory_usage as imu

imu.start_watching_memory()

In [41] used 0.0312 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 32.73 MiB


In [49]:
%%writefile slots.py
class MyClass(object):
    __slots__ = ['name','identifier']
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
        
num = 1024*256
x = [MyClass(1,1) for i in range(num)]

Writing slots.py
In [49] used 0.0625 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 59.23 MiB


In [50]:
from slots import *

In [50] used 1.2422 MiB RAM in 0.31s, peaked 8.34 MiB above current, total RAM usage 60.47 MiB


In [51]:
%%writefile noslots.py
class MyClass(object):
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
        
num = 1024*256
x = [MyClass(1,1) for i in range(num)]

Writing noslots.py
In [51] used -1.6797 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 58.79 MiB


In [52]:
from noslots import *

In [52] used 45.7969 MiB RAM in 0.37s, peaked 0.00 MiB above current, total RAM usage 104.59 MiB


## 3.11 Virtual Environment

`Virtualenv` is a tool for making isolated Python environments. Imagine an application that needs version 2 of a library, but another application requires version 3. If you install everything to the platform's standard location, it's easy to end up in a situation where a package is unintentionally upgraded.

`virtualenv` creates isolated environments for your Python application and allows you to install Python libraries in that isolated environment instead of installing them globally.

The most important commands are:
- `$ virtualenv myproject`
- `$ source myproject/bin/activate`

The first one makes an isolated `virtualenv` environment in the myproject folder and the second command activates that isolated environment.

While creating the virtualenv you have to make a decision. Do you want this `virtualenv` to use packages from your system `site-packages` or install them in the `virtualenv` `site-packages`? By default, `virtualenv` will not give access to the global `site-packages`.

If you want your `virtualenv` to have access to your systems `site-packages`, use the `--system-site-packages` switch when creating your `virtualenv` like this:

`$ virtualenv --system-site-packages mycoolproject`

You can turn off the `env` by typing:

`$ deactivate`

Running *python* after deactivating will use your system installation of Python again.

## 3.12 Collections

Python ships with a module that contains a number of container data types called Collections. We will talk about a few of them and discuss their usefulness:
- defaultdict
- OrderedDict
- counter
- deque
- namedtuple
- enum.Enum (outside of the module; Python 3.4+)

### 3.12.1 defaultdict

Unlike `dict`, with defaultdict you do not need to check whether a key is present or not. Example:

In [2]:
from collections import defaultdict

colours = (
    ('Yasoob', 'Yellow'),
    ('Ali', 'Blue'),
    ('Arham', 'Green'),
    ('Ali', 'Black'),
    ('Yasoob', 'Red'),
    ('Ahmed', 'Silver'),
)

favourite_colours = defaultdict(list)

for name, colour in colours:
    favourite_colours[name].append(colour)
    
print(favourite_colours)

defaultdict(<class 'list'>, {'Yasoob': ['Yellow', 'Red'], 'Ali': ['Blue', 'Black'], 'Arham': ['Green'], 'Ahmed': ['Silver']})


Appending nested lists inside a dictionary is an important use case. If a key is not already present in the dictionary, then you will receive a `KeyError`. `defaultdict` allows us to circumvent this issue in a clever way.

**Problem:**

In [3]:
some_dict = {}
some_dict['colours']['favourite'] = "yellow"

KeyError: 'colours'

**Solution:**

In [4]:
from collections import defaultdict
tree = lambda: defaultdict(tree)
some_dict = tree()
some_dict['colours']['favourite'] = "yellow"

In [5]:
import json
print(json.dumps(some_dict))

{"colours": {"favourite": "yellow"}}


### 3.12.2 OrderedDict

`OrderedDict` keeps its entries sorted as they are initially inserted. Overwriting a value of an existing key doesn't change the position of that key. However, deleting and reinserting an entry moves the key to the end of the dictionary.

**Problem:**

In [9]:
colours = {"Red": 198,"Green": 170,"Blue": 160}
for key, value in colours.items():
    print(key,value)

Red 198
Green 170
Blue 160


**Entries are supposed to be retrieved in an unpredictable order... here they happened to print in order.**

**Solution:**

In [10]:
from collections import OrderedDict

colours = OrderedDict([("Red", 198),("Green", 170), ("Blue", 160)])
for key, value in colours.items():
    print(key,value)

Red 198
Green 170
Blue 160


### 3.12.3 counter

Counter allows us to count the occurrences of a particular item. For instance it can be used to count the number of inidividual favourite colours:

In [11]:
from collections import Counter

colours = (
    ('Yasoob', 'Yellow'),
    ('Ali', 'Blue'),
    ('Arham', 'Green'),
    ('Ali', 'Black'),
    ('Yasoob', 'Red'),
    ('Ahmed', 'Silver'),
)

favs = Counter(name for name, colour in colours)
print(favs)

Counter({'Yasoob': 2, 'Ali': 2, 'Arham': 1, 'Ahmed': 1})


We can also count the most common lines in a file. For example:

In [12]:
with open('filename','rb') as f:
    line_count = Counter(f)
print(line_count)

FileNotFoundError: [Errno 2] No such file or directory: 'filename'